In [1]:
import pandas as pd
import urllib
import requests
import numpy as np
import os
import datetime
from bs4 import BeautifulSoup

# URLを取得し、解析する #
def get_and_bs_html(dic):
    html = requests.get(dic["URL"]+dic["Conditions"])
    if dic["Name"] == "zero-one":
        bs = BeautifulSoup(html.content,"html.parser")
    else:
        bs = BeautifulSoup(html.text,"html.parser")
    return bs

# 新着記事かどうかの判断基準を決める #
def select_criterion(dic,df):
    df_sorted_by_source = df[df["Source"]==dic["Name"]]
    criterion = list(df_sorted_by_source["URL"]) # urls_stored
    return criterion, df_sorted_by_source

# サイトによって異なるスクレイピング関数 #
def search_new_article(dic,df):
    # 前準備 #
    columns = ["Title","URL","Summary","Source","Date"]
    today = datetime.date.today().strftime("%Y/%m/%d")
    bs = get_and_bs_html(dic)
    criterion,df_sorted_by_source = select_criterion(dic,df)
    # 本番 #
    if dic["Name"] == "キャリアバイト": # キャリアバイト用
        all_articles = bs.find_all("a",{"class":"tapArea informClick"})
        for article in all_articles:
            avalibility = article.find("span",{"class":"newLabel"})
            if avalibility == None: # Newのラベルがなければbreak
                break
            else:
                url = dic["URL"]+article.get("href") # URLを特定
                if url not in criterion: # 新着記事であれば
                    title = article.find("h3",{"class":"summaryList_ttl"})
                    for e in title:
                        if (e.string != "NEW") & (len(e.string)>1):
                            title_ = e.string.strip("\n ") # タイトルを特定 
                        else:
                            continue
                    # summaryを特定 #
                    summary = article.find("div",{"class":"summaryList_lead"}).string
                    print(title_)
                    print(url)
                    print(summary)
                    se = pd.Series([title_,url,summary,"キャリアバイト",today],index=columns)
                    df = df.append(se,ignore_index=True)
                else:
                    continue
        # 新着記事がなければ #
        if df["Source"].value_counts()[dic["Name"]] == df_sorted_by_source["Source"].value_counts()[dic["Name"]]:
            print(dic["Name"]+"に新着求人はない")
            print(dic["URL"]+dic["Conditions"])
    if dic["Name"] == "Infra": # Infra用
        all_articles = bs.find_all("div",{"class":"col s12 m6"})
        for article in all_articles:
            title_and_url = article.find("h3",{"class":"intern-title"})
            # URLを特定 #
            url = dic["URL"]+title_and_url.find("a").get("href")
            # 新着記事であるかどうかを判断 #
            if url in criterion:
                break
            else:
                # タイトルを特定 #
                title = title_and_url.find("a").string.strip("\n ")
                # summaryを取得 #
                summary = article.find("td",{"class":"job-type"}).string
                print(title)
                print(url)
                print(summary)
                se = pd.Series([title,url,summary,"Infra",today],index=columns)
                df = df.append(se,ignore_index=True)      
                
        # 新着記事がなければ #
        if df["Source"].value_counts()[dic["Name"]] == df_sorted_by_source["Source"].value_counts()[dic["Name"]]:
            print(dic["Name"]+"に新着求人はない")
            print(dic["URL"]+dic["Conditions"])
    if dic["Name"] == "zero-one": # ゼロワン用
        all_articles = bs.find_all("section",{"class":"l-common-joblistBox"})
        for article in all_articles:
            avalibility = article.find("div",{"class":"l-common-joblist_fav_sp display-none_pc"}).find("span").string
            if avalibility != "お気に入り": # 募集中でなければbreak
                break
            else:
                title_and_url = article.find("div",{"class":"l-common-joblist_text"})
                # URLを取得 #
                url = dic["URL"] + title_and_url.find("a").get("href")
                # 募集中でありかつ新着であれば #
                if url not in criterion:
                    # タイトルを取得 #
                    title = title_and_url.find("h4").string.strip("\n ")
                    summary_and_loc = article.find_all("dl",{"class":"display-none_sp"})
                    # summaryを取得 #
                    summary = summary_and_loc[1].find("dd").string.strip("\n ")
                    print(title)
                    print(url)
                    print(summary)
                    se = pd.Series([title,url,summary,"zero-one",today],index=columns)
                    df = df.append(se,ignore_index=True)
                else:
                    continue
        # 新着記事がなければ #
        if df["Source"].value_counts()[dic["Name"]] == df_sorted_by_source["Source"].value_counts()[dic["Name"]]:
            print(dic["Name"]+"に新着求人はない")
            print(dic["URL"]+dic["Conditions"])
    if dic["Name"] == "Wantedly": # Wantedly用
        all_articles = bs.find_all("article",{"class":"projects-index-single"})
        for article in all_articles:
            url = dic["URL"]+article.find("h1",{"class":"project-title"}).find("a").get("href").split("?filter_params",1)[0]
            # 新着記事がなければ #
            if url in criterion:
                break
            # 新着記事であれば #
            else:
                # タイトルを特定
                title = article.find("h1",{"class":"project-title"}).find("a").string
                # summaryを取得 #
                summary = article.find("p",{"class":"project-excerpt"}).string.strip("\n ")
                print(title)
                print(url)
                print(summary)
                se = pd.Series([title,url,summary,"Wantedly",today],index=columns)
                df = df.append(se,ignore_index=True)
        # 新着記事がなければ #
        if df["Source"].value_counts()[dic["Name"]] == df_sorted_by_source["Source"].value_counts()[dic["Name"]]:
            print(dic["Name"]+"に新着求人はない")
            print(dic["URL"]+dic["Conditions"])
    return df
# サイトおよび条件の辞書 #
dic_career = {
    "Name":"キャリアバイト",
    "URL":"https://careerbaito.com",
    "Conditions":"/search/38/305/316"
}
dic_infra = {
    "Name":"Infra",
    "URL":"https://www.in-fra.jp/",
    "Conditions":"long-internships?occupation%5B%5D=1&occupation%5B%5D=7&area%5B%5D=1&area%5B%5D=4&area%5B%5D=5&area%5B%5D=2&industry%5B%5D=1&industry%5B%5D=4&industry%5B%5D=11&order=recent"
}
dic_zeroone = {
    "Name":"zero-one",
    "URL":"https://01intern.com",
    "Conditions":"/job/list.html?areas=13&jobTypes=6&jobTypes=2&businessTypes=1&businessTypes=4&businessTypes=8"
}
dic_wantedly = {
    "Name":"Wantedly",
    "URL":"https://www.wantedly.com/",
    "Conditions":"/projects?type=recent&page=1&occupation_types%5B%5D=jp__engineering&hiring_types%5B%5D=internship&fields%5B%5D=jp__data_scientist&locations%5B%5D=tokyo"
}
dic_list = [dic_career,dic_infra,dic_zeroone,dic_wantedly]

In [2]:
csv_path = os.path.join("datasets","Internship.csv")
df = pd.read_csv(csv_path,index_col=0)
df

,Title,URL,Summary,Source,Date
0,【技術で事業を創る】事業と一緒に成長したいエンジニアインターン！,https://careerbaito.com/job/32790,#東証一部上場連結子会社＃エンジニア＃WEB開発＃アプリ開発＃インフラ構築＃自然言語処理＃機械学習,キャリアバイト,2020/5/4
1,【経営コンサルタント社長直下】新規事業立ち上げ・BtoBマーケティング・営業企画のインターン,https://careerbaito.com/job/32562,＼新規事業立ち上げ／経営コンサルティングの企画・営業の裁量をすべてお任せます・自律的にジャッ...,キャリアバイト,2020/5/4
2,「美少女図鑑」WEBサイト制作インターン,https://www.in-fra.jp//long-internships/9742,WEBサイト制作,Infra,2020/5/4
3,フロントエンド／バックエンドエンジニア募集,https://www.in-fra.jp//long-internships/9728,フロントエンド／バックエンドエンジニア,Infra,2020/5/4
4,【未経験OK】営業・マーケ・CSを経験できるインターン！営業フリーランスを世の中に浸透させ業...,https://www.in-fra.jp//long-internships/9735,アウトバウンド営業 / マーケティング/CS,Infra,2020/5/4
...,...,...,...,...,...
81,N高等学校のインターン！高校生のプログラミング習得をサポートしよう,https://01intern.com/job/2672.html,ニコニコ動画を提供するIT企業ドワンゴと出版社KADOKAWAが創立したネットの学校「N高等...,zero-one,2020/06/17
82,起業・創業・ベンチャー中小企業経営者を支える財務・会計コンサルインターン,https://careerbaito.com/job/31047,ベンチャー中小企業経営者を財務・会計面から支えるコンサルタントを目指しませんか？本気で経営戦...,キャリアバイト,2020/06/18
83,優秀エンジニア多数在籍！スキルアップしたい学生注目の開発インターン,https://01intern.com/job/2388.html,株式会社サードスコープは、クリエイティブ×テクノロジーを軸にWebサービスの受託開発や自社サ...,zero-one,2020/06/19
84,インターン募集！AIプラットフォームを一緒に世に広めていくエンジニア募集！,https://www.wantedly.com//projects/470818,自社プラットフォーム「ENGER」を自社開発しています。このサービスは、HRTech分野での...,Wantedly,2020/06/19


In [6]:
for target in dic_list:
    df = search_new_article(target,df=df)

キャリアバイトに新着求人はない
https://careerbaito.com/search/38/305/316
Infraに新着求人はない
https://www.in-fra.jp/long-internships?occupation%5B%5D=1&occupation%5B%5D=7&area%5B%5D=1&area%5B%5D=4&area%5B%5D=5&area%5B%5D=2&industry%5B%5D=1&industry%5B%5D=4&industry%5B%5D=11&order=recent
zero-oneに新着求人はない
https://01intern.com/job/list.html?areas=13&jobTypes=6&jobTypes=2&businessTypes=1&businessTypes=4&businessTypes=8
Wantedlyに新着求人はない
https://www.wantedly.com//projects?type=recent&page=1&occupation_types%5B%5D=jp__engineering&hiring_types%5B%5D=internship&fields%5B%5D=jp__data_scientist&locations%5B%5D=tokyo


In [4]:
df.to_csv(csv_path,encoding='utf_8_sig')

In [5]:
df

,Title,URL,Summary,Source,Date
0,【技術で事業を創る】事業と一緒に成長したいエンジニアインターン！,https://careerbaito.com/job/32790,#東証一部上場連結子会社＃エンジニア＃WEB開発＃アプリ開発＃インフラ構築＃自然言語処理＃機械学習,キャリアバイト,2020/5/4
1,【経営コンサルタント社長直下】新規事業立ち上げ・BtoBマーケティング・営業企画のインターン,https://careerbaito.com/job/32562,＼新規事業立ち上げ／経営コンサルティングの企画・営業の裁量をすべてお任せます・自律的にジャッ...,キャリアバイト,2020/5/4
2,「美少女図鑑」WEBサイト制作インターン,https://www.in-fra.jp//long-internships/9742,WEBサイト制作,Infra,2020/5/4
3,フロントエンド／バックエンドエンジニア募集,https://www.in-fra.jp//long-internships/9728,フロントエンド／バックエンドエンジニア,Infra,2020/5/4
4,【未経験OK】営業・マーケ・CSを経験できるインターン！営業フリーランスを世の中に浸透させ業...,https://www.in-fra.jp//long-internships/9735,アウトバウンド営業 / マーケティング/CS,Infra,2020/5/4
...,...,...,...,...,...
83,優秀エンジニア多数在籍！スキルアップしたい学生注目の開発インターン,https://01intern.com/job/2388.html,株式会社サードスコープは、クリエイティブ×テクノロジーを軸にWebサービスの受託開発や自社サ...,zero-one,2020/06/19
84,インターン募集！AIプラットフォームを一緒に世に広めていくエンジニア募集！,https://www.wantedly.com//projects/470818,自社プラットフォーム「ENGER」を自社開発しています。このサービスは、HRTech分野での...,Wantedly,2020/06/19
85,AIなんて自分には…なんて思ってませんか？経験ゼロからでもじっくり育てます,https://www.wantedly.com//projects/468862,「最先端の技術に興味がある！」\n「だけど未経験の自分にはエンジニアなんて…」\n「ましてや...,Wantedly,2020/06/19
86,複数言語の習得を目指そう！リモートワークOKのエンジニアインターン,https://01intern.com/job/1058.html,就職前に色々な言語を覚えて自分のキャリアの選択肢を広げたい方におすすめのエンジニアインターン...,zero-one,2020/06/20
